In [3]:
import sys
!{sys.executable} -m pip install sqlite-utils

In [3]:
!wget http://db.csail.mit.edu/labdata/data.txt.gz
!gunzip data.txt.gz
!sed -i '1s/^/date time epoch moteid temperature humidity light voltage\n/' data.txt
!head data.txt

--2021-11-20 20:16:31--  http://db.csail.mit.edu/labdata/data.txt.gz
Resolving db.csail.mit.edu (db.csail.mit.edu)... 128.52.128.91
Connecting to db.csail.mit.edu (db.csail.mit.edu)|128.52.128.91|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34422518 (33M) [application/x-gzip]
Saving to: ‘data.txt.gz.2’

data.txt.gz.2       100%[===================>]  32.83M  90.5MB/s    in 0.4s    

2021-11-20 20:16:31 (90.5 MB/s) - ‘data.txt.gz.2’ saved [34422518/34422518]

date time epoch moteid temperature humidity light voltage
2004-03-31 03:38:15.757551 2 1 122.153 -3.91901 11.04 2.03397
2004-02-28 00:59:16.02785 3 1 19.9884 37.0933 45.08 2.69964
2004-02-28 01:03:16.33393 11 1 19.3024 38.4629 45.08 2.68742
2004-02-28 01:06:16.013453 17 1 19.1652 38.8039 45.08 2.68742
2004-02-28 01:06:46.778088 18 1 19.175 38.8379 45.08 2.69964
2004-02-28 01:08:45.992524 22 1 19.1456 38.9401 45.08 2.68742
2004-02-28 01:09:22.323858 23 1 19.1652 38.872 45.08 2.68742
2004-02-28 01:09:46.10

In [7]:
!sqlite-utils insert intel-sensor.sqlite readings data.txt --csv --sniff --detect-types

  [####################################]  100%          

In [9]:
!sqlite-utils schema intel-sensor.sqlite

CREATE TABLE "readings" (
   [date] TEXT,
   [time] TEXT,
   [epoch] INTEGER,
   [moteid] INTEGER,
   [temperature] FLOAT,
   [humidity] FLOAT,
   [light] FLOAT,
   [voltage] FLOAT
);


In [20]:
from sqlalchemy import create_engine
from datools.explanation.algorithms import diff
from datools.models import Column
from datools.sqlalchemy_utils import query_results_pretty_print

engine = create_engine('sqlite:///intel-sensor.sqlite')

query_results_pretty_print(
    engine, 
    'SELECT COUNT(*) FROM readings  WHERE temperature >= 100',
    'high temp')

query_results_pretty_print(
    engine, 
    'SELECT COUNT(*) FROM readings  WHERE temperature < 100',
    'low temp')


candidates = diff(
        engine,
        'SELECT moteid FROM readings WHERE temperature >= 100',
        'SELECT moteid FROM readings WHERE temperature < 100',
        (Column('moteid'),),
        0,
        2.0,
        1)
print(candidates)

*** high temp ***
+------------+
|   COUNT(*) |
|------------|
|     384344 |
+------------+
*** low temp ***
+------------+
|   COUNT(*) |
|------------|
|    1929338 |
+------------+
[Explanation(predicates=(Predicate(left=Column(name='moteid'), operator=<Operator.EQUALS: 1>, right=''),), risk_ratio=6.0266896636174865), Explanation(predicates=(Predicate(left=Column(name='moteid'), operator=<Operator.EQUALS: 1>, right=5),), risk_ratio=6.020267492388957), Explanation(predicates=(Predicate(left=Column(name='moteid'), operator=<Operator.EQUALS: 1>, right=57),), risk_ratio=6.01984950903102), Explanation(predicates=(Predicate(left=Column(name='moteid'), operator=<Operator.EQUALS: 1>, right=6485),), risk_ratio=6.019823387382137), Explanation(predicates=(Predicate(left=Column(name='moteid'), operator=<Operator.EQUALS: 1>, right=65407),), risk_ratio=6.019823387382137)]


In [ ]:
!rm data.txt* intel-sensor.sqlite*